<a href="https://colab.research.google.com/github/Tejnu/CSET-419---Generative-artificial-intelligence/blob/main/GENAI_4_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
from collections import defaultdict

In [ ]:

text = """
artificial intelligence is transforming modern society.
it is used in healthcare finance education and transportation.
machine learning allows systems to improve automatically with experience.
data plays a critical role in training intelligent systems.
large datasets help models learn complex patterns.
deep learning uses multi layer neural networks.
neural networks are inspired by biological neurons.
each neuron processes input and produces an output.
training a neural network requires optimization techniques.
gradient descent minimizes the loss function.

natural language processing helps computers understand human language.
text generation is a key task in nlp.
language models predict the next word or character.
recurrent neural networks handle sequential data.
lstm and gru models address long term dependency problems.
however rnn based models are slow for long sequences.

transformer models changed the field of nlp.
they rely on self attention mechanisms.
attention allows the model to focus on relevant context.
transformers process data in parallel.
this makes training faster and more efficient.
modern language models are based on transformers.

education is being improved using artificial intelligence.
intelligent tutoring systems personalize learning.
automated grading saves time for teachers.
online education platforms use recommendation systems.
technology enhances the quality of learning experiences.

ethical considerations are important in artificial intelligence.
fairness transparency and accountability must be ensured.
ai systems should be designed responsibly.
data privacy and security are major concerns.
researchers continue to improve ai safety.

text generation models can create stories poems and articles.
they are used in chatbots virtual assistants and content creation.
generated text should be meaningful and coherent.
evaluation of text generation is challenging.
human judgement is often required.

continuous learning is essential in the field of ai.
research and innovation drive technological progress.
students should build strong foundations in mathematics.
programming skills are important for ai engineers.
practical experimentation enhances understanding.

"""


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:

def preprocess_text(text):
    text = text.lower()
    text = text.replace('\n', ' ')
    tokens = text.split()
    return tokens

tokens = preprocess_text(text)

In [ ]:

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

total_words = len(tokenizer.word_index) + 1

token_list = tokenizer.texts_to_sequences([text])[0]

In [ ]:

input_sequences = []

for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

max_len = max(len(seq) for seq in input_sequences)

input_sequences = pad_sequences(input_sequences, maxlen=max_len, padding='pre')

X = input_sequences[:, :-1]
y = input_sequences[:, -1]

y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [ ]:

model = Sequential()
model.add(Embedding(total_words, 64, input_length=max_len-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
def sample_with_temperature(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)

def generate_text(seed_text, next_words, model, max_len, temperature=0.8):
    for _ in range(next_words):
        sequence = tokenizer.texts_to_sequences([seed_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_len-1, padding='pre')

        preds = model.predict(sequence, verbose=0)[0]
        predicted_index = sample_with_temperature(preds, temperature)

        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                seed_text += " " + word
                break

    return seed_text

In [ ]:
model.fit(X, y, epochs=150, verbose=1)

Epoch 1/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 291ms/step - accuracy: 0.0034 - loss: 5.2736
Epoch 2/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 172ms/step - accuracy: 0.0624 - loss: 5.2600
Epoch 3/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 170ms/step - accuracy: 0.0142 - loss: 5.2267
Epoch 4/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 168ms/step - accuracy: 0.0263 - loss: 5.0889
Epoch 5/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 179ms/step - accuracy: 0.0362 - loss: 5.0548
Epoch 6/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 169ms/step - accuracy: 0.0316 - loss: 5.0463
Epoch 7/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 234ms/step - accuracy: 0.0384 - loss: 4.9947
Epoch 8/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 169ms/step - accuracy: 0.0375 - loss: 4.9817
Epoch 9/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 171ms/step - accuracy: 0.0416 - loss: 4.9276
Epoch 10/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 174ms/step - accuracy: 0.0476 - loss: 4.8946
Epoch 11/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 171ms/step - accuracy: 0.0521 - loss: 4.7907
Epoch 12/150
10/10 ━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
# LSTM
seed_text = "artificial intelligence"
print(generate_text(seed_text, 20, model, max_len, temperature=0.6))
print(generate_text(seed_text, 20, model, max_len, temperature=0.8))
print(generate_text(seed_text, 20, model, max_len, temperature=1.0))

artificial intelligence is transforming modern society it is used in healthcare finance education and transportation machine systems to improve automatically experience data
artificial intelligence is transforming society it is used in healthcare finance education learning and transportation machine learning allows systems to improve with
artificial intelligence is transforming society it is used in healthcare healthcare and healthcare learning machine and machine learning allows systems to improve
